In [66]:
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import pandas_profiling

import time
start_time = time.time()
mbo_data = pd.read_csv("Desktop/bio/MBO_DATA.csv", names = ["AUFNR", "TANK_NO", "SAMP_NM", "ACC_SUGAR", "AGE", "OD", "AN", "FEED_TOT", "MBO_VOL"])



In [67]:
media_data = pd.read_csv("Desktop/bio/MEDIA_DATA.csv", names = ["AUFNR", "DATE", "JAR_VOL", "EQUIP_CD", "TANK_NO", "Media_ID", "HSM", "CSL", "[(NH4)2SO4]", "KH2PO4", "[d-Biotin]", "[Thiamine-HCL]", "[Ca-Pantothenic]", "Niacinamide", "Defoamer", "BM", "CM", "Sucrose", "CuSO4", "FeSO4", "MgSO4", "MnSO4", "ZnSO4", "Betaine", "H3PO4", "SOD", "Glucose"], parse_dates=["DATE"])


In [68]:
result_data = pd.read_csv("Desktop/bio/RESULT_DATA.csv", names = ["AUFNR", "DATE", "JAR_VOL", "EQUIP_CD", "TANK_NO", "SAMP_NM", "AGE", "OD", "AN", "LYS", "AMS", "SUGAR_FEED", "MBO_VOL"], parse_dates=["DATE"])

### 각 열의 항목 의미  
#### MBO_DATA  

	AUFNR	varchar(12) - 공정 번호  
	TANK_NO	varchar(2) - 탱크 번호  
	SAMP_NM	varchar(10) - 샘플 번호  
	ACC_SUGAR	float - 누적 당 공급 양
	AGE	float - 배양 시간  
	OD	float - optical density (흡광도)  
	AN	float - ammonimum nitrogen (Kjeldahl method로 측정)
	FEED_TOT	float - 총 Feed 양
	MBO_VOL	float - 정제 이송량/ 유량계 값/ working vol
    
#### MEDIA_DATA		
	AUFNR	varchar(100) - 공정번호
	DATE	datetime - 날짜
	JAR_VOL	varchar(10) - JAR 용량
	EQUIP_CD	varchar(10) - equipment(장비)
	TANK_NO	varchar(50) - 탱크 번호
	Media_ID	varchar(10) - 배지
	HSM	float - 대두박 + 황산처리
	CSL	float - CSL 원부재료
	[(NH4)2SO4]	float - 황산 암모니움
	KH2PO4	float - 인산 칼륨
	[d-Biotin]	float - 바이오틴
	[Thiamine-HCL]	float - 티아민
	[Ca-Pantothenic ]	float - 판토텐산
	Niacinamide	float - 니아신아마이드
	Defoamer	float - 소포제
	BM	float - complex media중 하나
	CM	float - complex media중 하나
	Sucrose	float - sucrose (종모 원재료)  
	CuSO4	float - 황산 구리  
	FeSO4	float -  황산 철 
	MgSO4	float - 황산 마그네슘  
	MnSO4	float - 황상 망간  
	ZnSO4	float - 황산 아연  
	Betaine	float - 베타인
	H3PO4	float - 인산
	SOD	float - 초과산화물 불균등화효소
	Glucose	float - 글루코스 (포도당)
#### RESULT_DATA		 
	AUFNR	varchar(50) - 공정 번호
	DATE	datetime  - 날짜
	JAR_VOL	varchar(50) - JAR 용량
	EQUIP_CD	varchar(50) - equipment(장비)
	TANK_NO	varchar(50) - 탱크 번호
	SAMP_NM	varchar(50) - 샘플 번호
	Age	float - 배양 시간
	OD	float - optical density (흡광도)
	[AN (g kg)]	float - ammonimum nitrogen (Kjeldahl method로 측정)
	[LYS (g L)]	float - Lys 양
	[AMS(L)]	float - ammonium sulfate
	[Sugar feed vol (L)]	float - 당 공급 양
	[MBO vol (L)]	float - 정제 이송량/ 유량계 값/ working vol

데이터 상의 오류  
- MBO_DATA에서 ACC_SUGAR 항목은 RESULT_DATA에서 Sugar feed vol 항목을 누적해온 값을 저장하는 항목이지만 오류 발견  

    ex) ["AUFNR"] == "FLY5E1810003" & ["TANK_NO"] == 3 에서 MBO_DATA에서는 샘플 8M과 9M의 ACC_SUGAR 값 변화가 없지만 RESULT_DATA에서는 8M, 9M 모두 Sugar feed vol이 0.24임  
      
      
- MBO_DATA에서의 MBO VOL과 RESULT_DATA에서의 MBO VOL 불일치. 오직 FBO 샘플일 때만 일치

# MBO_DATA, MEDIA_DATA, RESULT DATA 통합하기  
  
  
예측 타겟: 1M까지의 AGE?? 혹은 FBO의 AGE

null 값이 많음  
mbo의 null 값은 result에서 가져오거나 알맞게 fill  
예측 모델에서 result의 variable은 쓰지 않으니 result의 null 값은 무시

LYS가 NaN인 경우: 1. SAMP_NM = START 2. SAMP_NM = 2F  
1일 경우 -> 0으로 fill  
2일 경우 ->

In [69]:
mbo_revised = pd.read_csv("Desktop/bio/MBO_DATA.csv", names = ["AUFNR", "TANK_NO", "SAMP_NM", "ACC_SUGAR", "AGE", "OD", "AN", "FEED_TOT", "MBO_VOL"])

In [70]:

mbo_revised = mbo_revised.sort_values(["AUFNR","TANK_NO","SAMP_NM"])
mbo_revised = mbo_revised[(mbo_revised["SAMP_NM"] == "START") | (mbo_revised["SAMP_NM"] =="1F") |(mbo_revised["SAMP_NM"] =="2F") | (mbo_revised["SAMP_NM"] =="1M")]

target 값이 1M까지임으로 1M이후 데이터 삭제

In [71]:
mbo_revised = mbo_revised[(mbo_revised["SAMP_NM"] == "START") | (mbo_revised["SAMP_NM"] =="1F") |(mbo_revised["SAMP_NM"] =="2F") | (mbo_revised["SAMP_NM"] =="1M")]

MBO 파일과 Result 파일을 AUFNR, TANK_NO, SAMP_NM 기준으로 통합

In [72]:
merged_data = pd.merge(mbo_revised, result_data, on = ['AUFNR', 'TANK_NO', 'SAMP_NM'], how = 'left')
merged_data.sort_values(["AUFNR","TANK_NO","SAMP_NM"])

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y
0,FFL3J1901004,1,1F,0.000,1.566667,24.580000,1.160,0.000,NaN,2019-01-23,3,J,1.57,24.58,1.160,21.929000,0.20,0.000,13.200
1,FFL3J1901004,1,1M,6.000,8.833333,57.939999,0.491,6.300,NaN,2019-01-23,3,J,8.83,57.94,0.491,94.100000,0.30,3.600,19.800
2,FFL3J1901004,1,2F,2.600,5.666667,49.099998,1.170,2.600,NaN,2019-01-23,3,J,5.67,49.10,1.170,62.686727,0.30,2.400,15.900
3,FFL3J1901004,1,START,0.000,0.000000,15.940000,1.720,0.000,NaN,2019-01-23,3,J,0.00,15.94,1.720,0.000000,0.00,0.000,13.000
4,FFL3J1901004,2,1F,0.000,1.566667,24.780001,1.130,0.000,NaN,2019-01-23,3,J,1.57,24.78,1.130,21.655000,0.00,0.000,13.000
5,FFL3J1901004,2,1M,6.000,8.650000,59.639999,0.647,6.200,2.60,2019-01-23,3,J,8.65,59.64,0.647,94.437000,0.30,3.600,19.300
6,FFL3J1901004,2,2F,2.400,5.633333,47.500000,0.433,2.600,NaN,2019-01-23,3,J,5.63,47.50,0.433,63.391571,0.20,2.400,15.600
7,FFL3J1901004,2,START,0.000,0.000000,16.110001,1.640,0.000,NaN,2019-01-23,3,J,0.00,16.11,1.640,0.000000,0.00,0.000,13.000
8,FFL3J1901004,3,1F,0.000,1.600000,24.410000,1.090,0.000,NaN,2019-01-23,3,J,1.60,24.41,1.090,22.211000,0.00,0.000,13.000
9,FFL3J1901004,3,1M,6.000,9.366667,59.599998,0.984,6.000,2.60,2019-01-23,3,J,9.37,59.60,0.984,97.251000,0.00,3.600,19.300


Age 값의 NaN 값 처리

In [73]:
merged_data[(merged_data.isnull()['AGE_x'])]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y
19,FFL3J1901005,2,1F,NaN,NaN,23.400000,1.260,NaN,NaN,2019-01-30,3,J,1.38,23.40,1.260,22.685000,0.00,0.000,13.000
40,FFL3J1902001,3,START,NaN,NaN,21.230000,1.870,NaN,NaN,2019-02-14,3,J,0.00,21.23,1.870,0.000000,0.00,0.000,12.000
68,FFL3J1902003,4,START,NaN,NaN,17.110001,2.020,NaN,NaN,2019-02-27,3,J,0.00,17.11,2.020,0.000000,0.00,0.000,12.000
69,FFL3J1903001,1,1F,NaN,NaN,21.700001,1.270,NaN,NaN,2019-03-06,3,J,1.28,21.70,1.270,19.387000,0.00,0.000,13.000
72,FFL3J1903001,1,START,NaN,NaN,15.910000,1.790,NaN,NaN,2019-03-06,3,J,0.00,15.91,1.790,0.000000,0.00,0.000,13.000
76,FFL3J1903001,2,START,NaN,NaN,15.780000,1.790,NaN,NaN,2019-03-06,3,J,0.00,15.78,1.790,0.000000,0.00,0.000,13.000
78,FFL3J1903001,3,1M,6.000,NaN,51.400002,3.290,6.000,3.000,2019-03-06,3,J,8.65,51.40,3.290,90.389000,0.00,3.600,19.300
84,FFL3J1903001,4,START,NaN,NaN,15.870000,1.790,NaN,NaN,2019-03-06,3,J,0.00,15.87,1.790,0.000000,0.00,0.000,13.000
86,FFL3J1903002,1,1M,NaN,NaN,62.700001,2.120,NaN,2.300,2019-03-13,3,J,7.42,57.80,0.000,103.000000,0.00,3.850,18.000
88,FFL3J1903002,1,START,NaN,NaN,NaN,1.780,NaN,NaN,2019-03-13,3,J,0.00,15.91,0.000,0.000000,0.00,0.000,12.000


In [74]:
merged_data['AGE_x'].fillna(merged_data['AGE_y'], inplace= True)
merged_data[(merged_data.isnull()['AGE_x'])]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y


OD의 Null 값 채우기

In [75]:
temp = merged_data[(merged_data.isnull()['OD_x'])]

In [76]:
merged_data['OD_x'].fillna(merged_data['OD_y'], inplace= True)
merged_data[temp.columns]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y
0,FFL3J1901004,1,1F,0.000,1.566667,24.580000,1.160,0.000,NaN,2019-01-23,3,J,1.57,24.58,1.160,21.929000,0.20,0.000,13.200
1,FFL3J1901004,1,1M,6.000,8.833333,57.939999,0.491,6.300,NaN,2019-01-23,3,J,8.83,57.94,0.491,94.100000,0.30,3.600,19.800
2,FFL3J1901004,1,2F,2.600,5.666667,49.099998,1.170,2.600,NaN,2019-01-23,3,J,5.67,49.10,1.170,62.686727,0.30,2.400,15.900
3,FFL3J1901004,1,START,0.000,0.000000,15.940000,1.720,0.000,NaN,2019-01-23,3,J,0.00,15.94,1.720,0.000000,0.00,0.000,13.000
4,FFL3J1901004,2,1F,0.000,1.566667,24.780001,1.130,0.000,NaN,2019-01-23,3,J,1.57,24.78,1.130,21.655000,0.00,0.000,13.000
5,FFL3J1901004,2,1M,6.000,8.650000,59.639999,0.647,6.200,2.60,2019-01-23,3,J,8.65,59.64,0.647,94.437000,0.30,3.600,19.300
6,FFL3J1901004,2,2F,2.400,5.633333,47.500000,0.433,2.600,NaN,2019-01-23,3,J,5.63,47.50,0.433,63.391571,0.20,2.400,15.600
7,FFL3J1901004,2,START,0.000,0.000000,16.110001,1.640,0.000,NaN,2019-01-23,3,J,0.00,16.11,1.640,0.000000,0.00,0.000,13.000
8,FFL3J1901004,3,1F,0.000,1.600000,24.410000,1.090,0.000,NaN,2019-01-23,3,J,1.60,24.41,1.090,22.211000,0.00,0.000,13.000
9,FFL3J1901004,3,1M,6.000,9.366667,59.599998,0.984,6.000,2.60,2019-01-23,3,J,9.37,59.60,0.984,97.251000,0.00,3.600,19.300


ACC_SUGAR 값 채우기

In [77]:
merged_data[(merged_data.isnull()['ACC_SUGAR'])]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y
19,FFL3J1901005,2,1F,NaN,1.380000,23.400000,1.260,NaN,NaN,2019-01-30,3,J,1.38,23.40,1.260,22.685000,0.00,0.000,13.000
40,FFL3J1902001,3,START,NaN,0.000000,21.230000,1.870,NaN,NaN,2019-02-14,3,J,0.00,21.23,1.870,0.000000,0.00,0.000,12.000
45,FFL3J1902002,1,1F,NaN,1.000000,23.200001,NaN,NaN,NaN,2019-02-20,3,J,1.00,23.20,0.000,20.477000,0.00,0.000,12.000
48,FFL3J1902002,1,START,NaN,0.000000,17.100000,NaN,NaN,NaN,2019-02-20,3,J,0.00,17.10,0.000,0.000000,0.00,0.000,12.000
49,FFL3J1902002,2,1F,NaN,1.000000,22.299999,NaN,NaN,NaN,2019-02-20,3,J,1.00,22.30,0.000,20.221000,0.00,0.000,12.000
52,FFL3J1902002,2,START,NaN,0.000000,17.600000,NaN,NaN,NaN,2019-02-20,3,J,0.00,17.60,0.000,0.000000,0.00,0.000,12.000
65,FFL3J1902003,4,1F,NaN,1.066667,23.400000,1.490,NaN,NaN,2019-02-27,3,J,1.07,23.40,1.490,21.747000,0.00,0.000,12.000
68,FFL3J1902003,4,START,NaN,0.000000,17.110001,2.020,NaN,NaN,2019-02-27,3,J,0.00,17.11,2.020,0.000000,0.00,0.000,12.000
69,FFL3J1903001,1,1F,NaN,1.280000,21.700001,1.270,NaN,NaN,2019-03-06,3,J,1.28,21.70,1.270,19.387000,0.00,0.000,13.000
72,FFL3J1903001,1,START,NaN,0.000000,15.910000,1.790,NaN,NaN,2019-03-06,3,J,0.00,15.91,1.790,0.000000,0.00,0.000,13.000


In [78]:
merged_data[merged_data['SAMP_NM'] == 'START']['ACC_SUGAR']

3      0.0
7      0.0
11     0.0
15     0.0
18     0.0
22     0.0
26     0.0
29     0.0
33     0.0
37     0.0
40     NaN
44     0.0
48     NaN
52     NaN
56     0.0
60     0.0
64     0.0
68     NaN
72     NaN
76     NaN
80     0.0
84     NaN
88     NaN
92     0.0
96     NaN
100    NaN
104    NaN
108    NaN
112    0.0
116    0.0
      ... 
473    0.0
476    0.0
479    0.0
482    0.0
485    0.0
489    NaN
493    0.0
497    0.0
501    0.0
505    0.0
509    0.0
513    0.0
517    0.0
521    0.0
525    0.0
529    NaN
533    NaN
537    NaN
541    0.0
545    NaN
549    0.0
553    NaN
557    NaN
561    NaN
565    NaN
569    0.0
573    NaN
577    NaN
581    NaN
585    NaN
Name: ACC_SUGAR, Length: 149, dtype: float64

ACC_SUGAR 가 start에서 NaN일 경우 0으로 처리

In [79]:
con = merged_data['SAMP_NM'] == 'START'

merged_data.loc[con, 'ACC_SUGAR'] = merged_data.loc[con, 'ACC_SUGAR'].fillna(0)

SAMP_NM Mapping화

In [80]:
media_mapping = {'START' :1, '1F':2, '2F':3, '1M' : 4, '2M' :5 ,'3M':6, '4M':7, '5M':8, '6M':9, '7M':10, '8M':11, '9M':12, 'FBO':13}
for data in [merged_data]:
    data['SAMP_NM'] = data['SAMP_NM'].map(media_mapping)

In [81]:
merged_data = merged_data.sort_values([ "AUFNR","TANK_NO","SAMP_NM"])
merged_data.reset_index()
merged_data.head()

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y
3,FFL3J1901004,1,1,0.0,0.000000,15.940000,1.720,0.0,NaN,2019-01-23,3,J,0.00,15.94,1.720,0.000000,0.0,0.0,13.0
0,FFL3J1901004,1,2,0.0,1.566667,24.580000,1.160,0.0,NaN,2019-01-23,3,J,1.57,24.58,1.160,21.929000,0.2,0.0,13.2
2,FFL3J1901004,1,3,2.6,5.666667,49.099998,1.170,2.6,NaN,2019-01-23,3,J,5.67,49.10,1.170,62.686727,0.3,2.4,15.9
1,FFL3J1901004,1,4,6.0,8.833333,57.939999,0.491,6.3,NaN,2019-01-23,3,J,8.83,57.94,0.491,94.100000,0.3,3.6,19.8
7,FFL3J1901004,2,1,0.0,0.000000,16.110001,1.640,0.0,NaN,2019-01-23,3,J,0.00,16.11,1.640,0.000000,0.0,0.0,13.0


In [82]:
merged_data[merged_data.isna()['ACC_SUGAR']]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y
19,FFL3J1901005,2,2,NaN,1.380000,23.400000,1.260,NaN,NaN,2019-01-30,3,J,1.38,23.40,1.260,22.685000,0.00,0.000,13.000
45,FFL3J1902002,1,2,NaN,1.000000,23.200001,NaN,NaN,NaN,2019-02-20,3,J,1.00,23.20,0.000,20.477000,0.00,0.000,12.000
49,FFL3J1902002,2,2,NaN,1.000000,22.299999,NaN,NaN,NaN,2019-02-20,3,J,1.00,22.30,0.000,20.221000,0.00,0.000,12.000
65,FFL3J1902003,4,2,NaN,1.066667,23.400000,1.490,NaN,NaN,2019-02-27,3,J,1.07,23.40,1.490,21.747000,0.00,0.000,12.000
69,FFL3J1903001,1,2,NaN,1.280000,21.700001,1.270,NaN,NaN,2019-03-06,3,J,1.28,21.70,1.270,19.387000,0.00,0.000,13.000
73,FFL3J1903001,2,2,NaN,1.450000,21.700001,1.320,NaN,NaN,2019-03-06,3,J,1.45,21.70,1.320,19.423000,0.00,0.000,13.000
81,FFL3J1903001,4,2,NaN,1.300000,22.170000,1.410,NaN,NaN,2019-03-06,3,J,1.30,22.17,1.410,19.131000,0.30,0.000,13.300
85,FFL3J1903002,1,2,NaN,0.966667,22.860001,1.430,NaN,NaN,2019-03-13,3,J,0.97,21.70,0.000,22.000000,0.00,0.000,12.000
86,FFL3J1903002,1,4,NaN,7.420000,62.700001,2.120,NaN,2.300,2019-03-13,3,J,7.42,57.80,0.000,103.000000,0.00,3.850,18.000
93,FFL3J1903002,3,2,NaN,1.083333,22.400000,1.440,0.30,NaN,2019-03-13,3,J,1.28,22.58,1.440,19.825000,0.30,0.000,12.300


In [83]:
for index, data in merged_data.iterrows():
    if data.isnull()['ACC_SUGAR']:
        if data['SAMP_NM'] == 2: #1F
            prev_acc_sugar = merged_data[(merged_data["AUFNR"] == data["AUFNR"]) & (merged_data["TANK_NO"] == data["TANK_NO"]) & (merged_data["SAMP_NM"] == 1)]["ACC_SUGAR"]
            
        elif data['SAMP_NM'] == 3:
            prev_acc_sugar = merged_data[(merged_data["AUFNR"] == data["AUFNR"]) & (merged_data["TANK_NO"] == data["TANK_NO"]) & (merged_data["SAMP_NM"] == 2)]["ACC_SUGAR"]
     
        elif data['SAMP_NM'] == 4:
            prev_acc_sugar = merged_data[(merged_data["AUFNR"] == data["AUFNR"]) & (merged_data["TANK_NO"] == data["TANK_NO"]) & (merged_data["SAMP_NM"] == 3)]["ACC_SUGAR"]
        
        if data.isnull()['SUGAR_FEED']:
            merged_data.at[index,'ACC_SUGAR'] = float(prev_acc_sugar)
        else:
            merged_data.at[index, 'ACC_SUGAR'] = float(prev_acc_sugar) + data['SUGAR_FEED']
              
merged_data[(merged_data.isnull()['ACC_SUGAR'])]


,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y


In [84]:
merged_data[(merged_data['AUFNR']=='FLY5E1811002') &(merged_data['TANK_NO']== 2)]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE,JAR_VOL,EQUIP_CD,AGE_y,OD_y,AN_y,LYS,AMS,SUGAR_FEED,MBO_VOL_y
553,FLY5E1811002,2,1,0.000,0.000000,11.550000,3.62,NaN,NaN,2018-11-14,5,E,0.00,11.55,3.62,0.000000,0.0,0.000,1.800
550,FLY5E1811002,2,2,0.000,7.083333,43.090000,1.34,NaN,NaN,2018-11-14,5,E,7.08,43.09,1.34,48.561000,0.0,0.000,1.800
552,FLY5E1811002,2,3,0.376,11.080000,63.700001,1.48,0.376,NaN,2018-11-14,5,E,11.08,63.70,1.48,100.714053,0.0,0.375,2.175
551,FLY5E1811002,2,4,0.800,14.450000,74.500000,1.75,0.800,0.24,2018-11-14,5,E,14.45,74.50,1.75,144.653000,0.0,0.425,2.600


In [85]:
media_data.rename(columns={"Media_ID":"SAMP_NM"}, inplace = True)
media_data

media_mapping = {'F1' :2, 'F2':3, 'Main':4}
for data in [media_data]:
    data['SAMP_NM'] = data['SAMP_NM'].map(media_mapping)
media_data

,AUFNR,DATE,JAR_VOL,EQUIP_CD,TANK_NO,SAMP_NM,HSM,CSL,[(NH4)2SO4],KH2PO4,...,Sucrose,CuSO4,FeSO4,MgSO4,MnSO4,ZnSO4,Betaine,H3PO4,SOD,Glucose
0,FLY5E1810002,2018-10-10,5,E,1,4,0.00,11.700,16.000000,0.20000,...,49.746193,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
1,FLY5E1810002,2018-10-10,5,E,1,2,0.00,6.840,45.510500,0.64480,...,475.691290,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
2,FLY5E1810002,2018-10-10,5,E,1,3,0.00,6.840,34.611375,0.64480,...,475.691290,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
3,FLY5E1810002,2018-10-10,5,E,2,4,0.00,11.700,16.000000,0.20000,...,49.746193,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
4,FLY5E1810002,2018-10-10,5,E,2,2,0.00,3.420,22.755250,0.32240,...,237.845645,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
5,FLY5E1810002,2018-10-10,5,E,2,3,0.00,3.420,17.305688,0.32240,...,237.845645,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
6,FLY5E1810002,2018-10-10,5,E,3,4,0.00,11.700,16.000000,0.20000,...,49.746193,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
7,FLY5E1810002,2018-10-10,5,E,3,2,0.00,1.710,22.755250,0.16120,...,118.922823,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
8,FLY5E1810002,2018-10-10,5,E,3,3,0.00,1.710,17.305688,0.16120,...,118.922823,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0
9,FLY5E1810002,2018-10-10,5,E,4,4,0.00,11.700,16.000000,0.20000,...,49.746193,0.00,0.0,0.000,0.00,0.00,0.0,0.00,0.0,0.0


In [86]:
media_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447 entries, 0 to 446
Data columns (total 27 columns):
AUFNR               447 non-null object
DATE                447 non-null datetime64[ns]
JAR_VOL             447 non-null int64
EQUIP_CD            447 non-null object
TANK_NO             447 non-null int64
SAMP_NM             447 non-null int64
HSM                 447 non-null float64
CSL                 447 non-null float64
[(NH4)2SO4]         447 non-null float64
KH2PO4              447 non-null float64
[d-Biotin]          447 non-null float64
[Thiamine-HCL]      447 non-null float64
[Ca-Pantothenic]    447 non-null float64
Niacinamide         447 non-null float64
Defoamer            447 non-null float64
BM                  447 non-null float64
CM                  447 non-null float64
Sucrose             447 non-null float64
CuSO4               447 non-null float64
FeSO4               447 non-null float64
MgSO4               447 non-null float64
MnSO4               447 non-null fl

Media data와 통합  



In [87]:
all_merged = pd.merge(merged_data, media_data, on = ['AUFNR', 'TANK_NO', 'SAMP_NM'], how = 'left')
all_merged.drop(['DATE_y','AGE_y','OD_y', 'AMS', 'SUGAR_FEED', 'DATE_y', 'JAR_VOL_y','EQUIP_CD_y'], axis=1, inplace= True)


con = all_merged['SAMP_NM'] == '1'
media = ['HSM','CSL','[(NH4)2SO4]','KH2PO4','[d-Biotin]','[Thiamine-HCL]','[Ca-Pantothenic]','Niacinamide','Defoamer','BM','CM','Sucrose','CuSO4','FeSO4','MgSO4','MnSO4','ZnSO4','Betaine','H3PO4','SOD','Glucose']
all_merged[media] = all_merged[media].fillna(0)

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


MBO_VOL

In [88]:
all_merged[(all_merged.notnull()['MBO_VOL_x']) & (all_merged['SAMP_NM'] != 4)]
# 1M이 아닌 경우 0으로 처리해야함

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE_x,...,Sucrose,CuSO4,FeSO4,MgSO4,MnSO4,ZnSO4,Betaine,H3PO4,SOD,Glucose
98,FFL3J1903002,4,2,0.000,1.083333,21.799999,1.45,0.101,0.00,2019-03-13,...,358.668367,0.07,27.6,0.128243,12.32,0.55,0.0,0.19,0.0,0.0
234,FFL3J1905003,1,2,0.000,1.416667,25.100000,0.00,0.000,0.00,2019-05-12,...,343.900000,0.07,27.6,0.130000,12.32,0.55,0.0,0.19,0.0,0.0
238,FFL3J1905003,2,2,0.000,1.416667,25.200001,0.00,0.000,0.00,2019-05-12,...,343.900000,0.07,27.6,0.130000,12.32,0.55,0.0,0.19,0.0,0.0
282,FFL3J1906003,1,2,0.000,1.600000,19.700001,0.00,0.000,0.00,2019-06-12,...,343.900000,0.07,27.6,0.130000,12.32,0.55,0.0,0.19,0.0,0.0
354,FFL5E1904005,2,2,0.000,1.650000,26.400000,2.18,0.000,0.00,2019-05-01,...,343.900000,0.07,27.6,0.130000,12.32,0.55,0.0,0.19,0.0,0.0
358,FFL5E1904005,3,2,0.000,1.650000,25.660000,2.13,0.000,0.00,2019-05-01,...,343.900000,0.07,27.6,0.130000,12.32,0.55,0.0,0.19,0.0,0.0
559,FLY5E1811002,4,2,0.000,7.080000,42.480000,1.52,NaN,0.00,2018-11-14,...,468.270000,0.00,0.0,0.449000,0.00,0.00,0.0,0.00,0.0,0.0
575,FLY5E1811003,3,2,0.000,5.433333,51.669998,0.20,0.040,0.00,2018-11-21,...,468.270000,0.00,0.0,0.449000,0.00,0.00,0.0,0.00,0.0,0.0
580,FLY5E1811003,4,3,0.375,9.333333,72.800003,2.31,0.415,0.24,2018-11-21,...,363.312500,0.00,0.0,0.449000,0.00,0.00,0.0,0.00,0.0,0.0


In [89]:
con = all_merged['SAMP_NM'] != 4

all_merged.loc[con, 'MBO_VOL_x'] = all_merged.loc[con, 'MBO_VOL_x'].fillna(0)

all_merged[all_merged.isnull()['MBO_VOL_x']]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE_x,...,Sucrose,CuSO4,FeSO4,MgSO4,MnSO4,ZnSO4,Betaine,H3PO4,SOD,Glucose
3,FFL3J1901004,1,4,6.0,8.833333,57.939999,0.491,6.30,NaN,2019-01-23,...,10.710000,0.0,0.0,0.23,0.0,0.0,0.0,0.29,0.0,0.0
256,FFL3J1906001,2,4,7.3,8.400000,46.900002,NaN,7.60,NaN,2019-05-31,...,10.710000,0.0,0.0,0.23,0.0,0.0,0.0,0.29,0.0,0.0
569,FLY5E1811003,1,4,0.0,14.333333,68.650002,1.620,0.00,NaN,2018-11-21,...,49.746193,0.0,0.0,0.37,0.0,0.0,0.0,0.00,0.0,0.0
573,FLY5E1811003,2,4,0.8,12.670000,79.809998,1.580,NaN,NaN,2018-11-21,...,49.746193,0.0,0.0,0.37,0.0,0.0,0.0,0.00,0.0,0.0
577,FLY5E1811003,3,4,0.0,15.883333,67.610001,4.520,0.04,NaN,2018-11-21,...,49.746193,0.0,0.0,0.37,0.0,0.0,0.0,0.00,0.0,0.0


In [90]:
all_merged[(all_merged.isnull()['MBO_VOL_x'])]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE_x,...,Sucrose,CuSO4,FeSO4,MgSO4,MnSO4,ZnSO4,Betaine,H3PO4,SOD,Glucose
3,FFL3J1901004,1,4,6.0,8.833333,57.939999,0.491,6.30,NaN,2019-01-23,...,10.710000,0.0,0.0,0.23,0.0,0.0,0.0,0.29,0.0,0.0
256,FFL3J1906001,2,4,7.3,8.400000,46.900002,NaN,7.60,NaN,2019-05-31,...,10.710000,0.0,0.0,0.23,0.0,0.0,0.0,0.29,0.0,0.0
569,FLY5E1811003,1,4,0.0,14.333333,68.650002,1.620,0.00,NaN,2018-11-21,...,49.746193,0.0,0.0,0.37,0.0,0.0,0.0,0.00,0.0,0.0
573,FLY5E1811003,2,4,0.8,12.670000,79.809998,1.580,NaN,NaN,2018-11-21,...,49.746193,0.0,0.0,0.37,0.0,0.0,0.0,0.00,0.0,0.0
577,FLY5E1811003,3,4,0.0,15.883333,67.610001,4.520,0.04,NaN,2018-11-21,...,49.746193,0.0,0.0,0.37,0.0,0.0,0.0,0.00,0.0,0.0


AN의 NaN 값 처리

In [91]:
all_merged[(all_merged.isnull()['AN_x'])]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE_x,...,Sucrose,CuSO4,FeSO4,MgSO4,MnSO4,ZnSO4,Betaine,H3PO4,SOD,Glucose
22,FFL3J1901005,2,4,6.000,8.716666,58.900002,NaN,6.000,2.60,2019-01-30,...,10.710000,0.00,0.0,0.230,0.00,0.00,0.0,0.29,0.0,0.000000
26,FFL3J1901005,3,4,6.000,8.700000,58.200001,NaN,6.300,2.60,2019-01-30,...,10.710000,0.00,0.0,0.230,0.00,0.00,0.0,0.29,0.0,0.000000
31,FFL3J1902001,1,2,0.000,1.333333,26.400000,NaN,0.000,0.00,2019-02-14,...,363.581633,0.07,27.6,0.130,12.32,0.55,0.0,0.19,0.0,0.000000
32,FFL3J1902001,1,3,3.200,6.033333,54.110001,NaN,3.200,0.00,2019-02-14,...,285.051020,0.07,27.6,0.130,12.32,0.55,0.0,0.19,0.0,81.318681
35,FFL3J1902001,2,2,0.000,1.433333,24.200001,NaN,0.000,0.00,2019-02-14,...,363.581633,0.07,27.6,0.130,12.32,0.55,0.0,0.19,0.0,0.000000
36,FFL3J1902001,2,3,3.200,6.283333,53.880001,NaN,3.200,0.00,2019-02-14,...,285.051020,0.07,27.6,0.130,12.32,0.55,0.0,0.19,0.0,81.318681
39,FFL3J1902001,3,2,0.000,1.333333,27.299999,NaN,0.000,0.00,2019-02-14,...,363.581633,0.07,27.6,0.130,12.32,0.55,0.0,0.19,0.0,0.000000
40,FFL3J1902001,3,3,3.200,6.533333,54.540001,NaN,3.200,0.00,2019-02-14,...,285.051020,0.07,27.6,0.130,12.32,0.55,0.0,0.19,0.0,81.318681
42,FFL3J1902001,4,2,0.000,1.333333,28.600000,NaN,0.000,0.00,2019-02-14,...,363.581633,0.07,27.6,0.130,12.32,0.55,0.0,0.19,0.0,0.000000
43,FFL3J1902001,4,3,3.200,6.550000,52.880001,NaN,3.200,0.00,2019-02-14,...,285.051020,0.07,27.6,0.130,12.32,0.55,0.0,0.19,0.0,81.318681


In [92]:
all_merged[(all_merged.isnull()['AN_x']) & (all_merged.isnull()['AN_y'])]
all_merged['AN_x'].fillna(all_merged['AN_y'], inplace= True)
all_merged[(all_merged.isnull()['AN_x'])]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE_x,OD_x,AN_x,FEED_TOT,MBO_VOL_x,DATE_x,...,Sucrose,CuSO4,FeSO4,MgSO4,MnSO4,ZnSO4,Betaine,H3PO4,SOD,Glucose
159,FFL3J1904002,1,3,2.260,5.416667,41.200001,NaN,NaN,0.0,2019-04-10,...,285.1,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
163,FFL3J1904002,2,3,2.260,5.500000,40.299999,NaN,NaN,0.0,2019-04-10,...,285.1,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
167,FFL3J1904002,3,3,2.260,5.583333,41.000000,NaN,NaN,0.0,2019-04-10,...,285.1,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
171,FFL3J1904002,4,3,2.260,5.750000,40.700001,NaN,NaN,0.0,2019-04-10,...,285.1,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
175,FFL3J1904002,5,3,2.260,5.716667,40.900002,NaN,NaN,0.0,2019-04-10,...,285.1,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
199,FFL3J1904004,2,3,2.260,5.500000,45.000000,NaN,NaN,0.0,2019-04-24,...,285.1,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
203,FFL3J1904004,3,3,2.260,5.416667,45.299999,NaN,2.260,0.0,2019-04-24,...,285.1,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
207,FFL3J1904004,4,3,2.260,5.250000,45.299999,NaN,2.260,0.0,2019-04-24,...,285.1,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
351,FFL5E1904005,1,3,0.225,5.433333,43.040001,NaN,0.225,0.0,2019-05-01,...,269.6,0.07,27.6,0.13,12.32,0.05,0.0,0.19,0.0,76.9
355,FFL5E1904005,2,3,0.225,5.366667,43.110001,NaN,0.225,0.0,2019-05-01,...,269.6,0.07,27.6,0.13,12.32,0.05,0.0,0.19,0.0,76.9


In [93]:
all_merged.drop(['AN_y','MBO_VOL_y'], axis=1, inplace= True)

In [94]:
all_merged.rename(columns={"AGE_x":"AGE", "OD_x":"OD", "AN_x":"AN", "MBO_VOL_x":"MBO_VOL", "DATE_x":"DATE","JAR_VOL_x":"JAR_VOL","EQUIP_CD_x":"EQUIP_CD"}, inplace = True)
all_merged[['AUFNR','TANK_NO','SAMP_NM','ACC_SUGAR','AGE','OD','AN','FEED_TOT','MBO_VOL']]

,AUFNR,TANK_NO,SAMP_NM,ACC_SUGAR,AGE,OD,AN,FEED_TOT,MBO_VOL
0,FFL3J1901004,1,1,0.000,0.000000,15.940000,1.720,0.000,0.00
1,FFL3J1901004,1,2,0.000,1.566667,24.580000,1.160,0.000,0.00
2,FFL3J1901004,1,3,2.600,5.666667,49.099998,1.170,2.600,0.00
3,FFL3J1901004,1,4,6.000,8.833333,57.939999,0.491,6.300,NaN
4,FFL3J1901004,2,1,0.000,0.000000,16.110001,1.640,0.000,0.00
5,FFL3J1901004,2,2,0.000,1.566667,24.780001,1.130,0.000,0.00
6,FFL3J1901004,2,3,2.400,5.633333,47.500000,0.433,2.600,0.00
7,FFL3J1901004,2,4,6.000,8.650000,59.639999,0.647,6.200,2.60
8,FFL3J1901004,3,1,0.000,0.000000,16.209999,1.680,0.000,0.00
9,FFL3J1901004,3,2,0.000,1.600000,24.410000,1.090,0.000,0.00


In [95]:
media_mapping = {'START' :1, '1F':2, '2F':3, '1M' : 4, '2M' :5 ,'3M':6, '4M':7, '5M':8, '6M':9, '7M':10, '8M':11, '9M':12, 'FBO':13}
for data in [result_data]:
    data['SAMP_NM'] = data['SAMP_NM'].map(media_mapping)
    


In [96]:
all_merged.drop('LYS', axis =1, inplace= True)

In [97]:
all_merged.drop('FEED_TOT', axis =1, inplace= True)

같은 공정 데이터 하나의 열로 통합

In [98]:
start = all_merged[all_merged['SAMP_NM'] == 1]
one_f= all_merged[all_merged['SAMP_NM'] == 2]
two_f= all_merged[all_merged['SAMP_NM'] == 3]
one_m= all_merged[all_merged['SAMP_NM'] == 4]

In [99]:
start.drop( ['AGE','ACC_SUGAR','MBO_VOL','DATE','EQUIP_CD','HSM','CSL','[(NH4)2SO4]','KH2PO4','[d-Biotin]','[Thiamine-HCL]','[Ca-Pantothenic]','Niacinamide','Defoamer','BM','CM','Sucrose','CuSO4','FeSO4','MgSO4','MnSO4','ZnSO4','Betaine','H3PO4','SOD','Glucose'], axis=1, inplace=True)
start.head()

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,AUFNR,TANK_NO,SAMP_NM,OD,AN,JAR_VOL
0,FFL3J1901004,1,1,15.940000,1.72,3
4,FFL3J1901004,2,1,16.110001,1.64,3
8,FFL3J1901004,3,1,16.209999,1.68,3
12,FFL3J1901004,4,1,15.820000,1.62,3
16,FFL3J1901005,1,1,16.299999,1.82,3


In [100]:
one_f.drop(['DATE','ACC_SUGAR','MBO_VOL', 'EQUIP_CD','JAR_VOL','HSM','CSL','[d-Biotin]','[Thiamine-HCL]','[Ca-Pantothenic]','Niacinamide','Defoamer','CuSO4','FeSO4','MgSO4','MnSO4','ZnSO4','H3PO4','SOD','Glucose'], axis=1,inplace=True)
two_f.drop(['DATE','MBO_VOL','EQUIP_CD','JAR_VOL','HSM','CSL','[d-Biotin]','[Thiamine-HCL]','[Ca-Pantothenic]','Niacinamide','Defoamer','CuSO4','FeSO4','MgSO4','MnSO4','ZnSO4','H3PO4'], axis=1,inplace=True)
one_m.drop(['DATE','SOD', 'EQUIP_CD','JAR_VOL','HSM','CSL','[d-Biotin]','[Thiamine-HCL]','[Ca-Pantothenic]','Niacinamide','Defoamer','CuSO4','FeSO4','MgSO4','MnSO4','ZnSO4','H3PO4','SOD','Glucose'], axis=1,inplace=True)

In [101]:
start_1f = pd.merge(start,one_f, on=['AUFNR', 'TANK_NO'], how='left')
start_1f.rename(columns={"SAMP_NM_x":"SAMP_NM_START","OD_x":"OD_START","AN_x":"AN_START","SAMP_NM_y":"SAMP_NM_1F","OD_y":"OD_1F","AN_y":"AN_1F"}, inplace = True)
start_1f_2f = pd.merge(start_1f,two_f, on=['AUFNR', 'TANK_NO'],how='left')
start_1f_2f.rename(columns={"AGE_x":"AGE_1F",'[(NH4)2SO4]_x':'(NH4)2SO4_1F',"KH2PO4_x":"KH2PO4_1F","BM_x":"BM_1F","CM_x":"CM_1F","Sucrose_x":"Sucrose_1F","Betaine_x":"Betaine_1F","AGE_y":"AGE_2F",'[(NH4)2SO4]_y':'(NH4)2SO4_2F',"KH2PO4_y":"KH2PO4_2F","BM_y":"BM_2F","CM_y":"CM_2F","Sucrose_y":"Sucrose_2F","Betaine_y":"Betaine_2F","OD":"OD_2F","AN":"AN_2F"}, inplace = True)
start_1f_2f_1m = pd.merge(start_1f_2f,one_m, on=['AUFNR', 'TANK_NO'],how='left')
start_1f_2f_1m.rename(columns={"AGE":"AGE_1M",'[(NH4)2SO4]':'(NH4)2SO4_1M',"KH2PO4":"KH2PO4_1M","BM":"BM_1M","CM":"CM_1M","Sucrose":"Sucrose_1M","Betaine":"Betaine_1M","SAMP_NM_x":"SAMP_NM_2F","ACC_SUGAR_x":"ACC_SUGAR_2F","SAMP_NM_y":"SAMP_NM_1M","ACC_SUGAR_y":"ACC_SUGAR_1M","OD":"OD_1M","AN":"AN_1M"}, inplace = True)

start_1f_2f_1m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 148
Data columns (total 41 columns):
AUFNR            149 non-null object
TANK_NO          149 non-null int64
SAMP_NM_START    149 non-null int64
OD_START         149 non-null float64
AN_START         149 non-null float64
JAR_VOL          149 non-null int64
SAMP_NM_1F       149 non-null int64
AGE_1F           149 non-null float64
OD_1F            149 non-null float64
AN_1F            149 non-null float64
(NH4)2SO4_1F     149 non-null float64
KH2PO4_1F        149 non-null float64
BM_1F            149 non-null float64
CM_1F            149 non-null float64
Sucrose_1F       149 non-null float64
Betaine_1F       149 non-null float64
SAMP_NM_2F       149 non-null int64
ACC_SUGAR_2F     149 non-null float64
AGE_2F           149 non-null float64
OD_2F            149 non-null float64
AN_2F            136 non-null float64
(NH4)2SO4_2F     149 non-null float64
KH2PO4_2F        149 non-null float64
BM_2F            149 non-null fl

In [102]:
result_revised = result_data[ (result_data["SAMP_NM"] ==2)]
result_revised.drop(["AGE","SAMP_NM","DATE","JAR_VOL","EQUIP_CD","OD","AN","LYS","SUGAR_FEED","MBO_VOL","AMS"],axis=1,inplace= True)

result_added1 = pd.merge(start_1f_2f_1m, result_revised, on=['AUFNR', 'TANK_NO'])

result_revised = result_data[ (result_data["SAMP_NM"] ==3)]
result_revised.drop(["AGE","SAMP_NM","DATE","JAR_VOL","EQUIP_CD","OD","AN","LYS","SUGAR_FEED","MBO_VOL","AMS"],axis=1,inplace= True)


result_added2 = pd.merge(result_added1, result_revised, on=['AUFNR', 'TANK_NO'])

result_revised = result_data[ (result_data["SAMP_NM"] ==3)]
result_revised.drop(["AGE","SAMP_NM","DATE","JAR_VOL","EQUIP_CD","OD","AN","LYS","SUGAR_FEED","MBO_VOL","AMS"],axis=1,inplace= True)


result_added3 = pd.merge(result_added2, result_revised, on=['AUFNR', 'TANK_NO'])
result_revised = result_data[ (result_data["SAMP_NM"] ==13)]
result_revised.drop(["AMS","SAMP_NM","DATE","JAR_VOL","EQUIP_CD","OD","AN","LYS","SUGAR_FEED","MBO_VOL"],axis=1,inplace= True)

df = pd.merge(result_added3, result_revised, on=['AUFNR', 'TANK_NO'])
df.rename(columns={"AGE":"FBO_AGE"}, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 148
Data columns (total 42 columns):
AUFNR            149 non-null object
TANK_NO          149 non-null int64
SAMP_NM_START    149 non-null int64
OD_START         149 non-null float64
AN_START         149 non-null float64
JAR_VOL          149 non-null int64
SAMP_NM_1F       149 non-null int64
AGE_1F           149 non-null float64
OD_1F            149 non-null float64
AN_1F            149 non-null float64
(NH4)2SO4_1F     149 non-null float64
KH2PO4_1F        149 non-null float64
BM_1F            149 non-null float64
CM_1F            149 non-null float64
Sucrose_1F       149 non-null float64
Betaine_1F       149 non-null float64
SAMP_NM_2F       149 non-null int64
ACC_SUGAR_2F     149 non-null float64
AGE_2F           149 non-null float64
OD_2F            149 non-null float64
AN_2F            136 non-null float64
(NH4)2SO4_2F     149 non-null float64
KH2PO4_2F        149 non-null float64
BM_2F            149 non-null fl

In [103]:
media_data[(media_data['AUFNR'] == 'FFL3J1901005') & (media_data['TANK_NO'] == 1)]

,AUFNR,DATE,JAR_VOL,EQUIP_CD,TANK_NO,SAMP_NM,HSM,CSL,[(NH4)2SO4],KH2PO4,...,Sucrose,CuSO4,FeSO4,MgSO4,MnSO4,ZnSO4,Betaine,H3PO4,SOD,Glucose
180,FFL3J1901005,2019-01-30,3,J,1,4,8.90,0.0,4.05,0.0,...,10.71,0.00,0.0,0.23,0.00,0.00,0.0,0.29,0.0,0.0
181,FFL3J1901005,2019-01-30,3,J,1,2,3.36,0.0,41.23,0.0,...,363.60,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,0.0
182,FFL3J1901005,2019-01-30,3,J,1,3,3.36,0.0,80.00,0.0,...,285.10,0.07,27.6,0.13,12.32,0.55,0.0,0.19,0.0,81.3


결측치 다른 데이터 셋에서 가져오기

In [104]:
for index, data in df.iterrows():
    if data.isnull()['SAMP_NM_1M']:
        aufnr = data["AUFNR"]
        tank = data["TANK_NO"]
        
        age = result_data[(result_data["AUFNR"] == aufnr) & (result_data["TANK_NO"] == tank) & (result_data["SAMP_NM"] == 4)]["AGE"]
        od = result_data[(result_data["AUFNR"] == aufnr) & (result_data["TANK_NO"] == tank) & (result_data["SAMP_NM"] == 4)]["OD"]
        an = result_data[(result_data["AUFNR"] == aufnr) & (result_data["TANK_NO"] == tank) & (result_data["SAMP_NM"] == 4)]["AN"]
        
        prev_sug=df[(df["AUFNR"] == aufnr) & (df["TANK_NO"] == tank)]["ACC_SUGAR_2F"]
        sug = result_data[(result_data["AUFNR"] == aufnr) & (result_data["TANK_NO"] == tank) & (result_data["SAMP_NM"] == 4)]["SUGAR_FEED"]
        
        so4 = media_data[(media_data["AUFNR"] == aufnr) & (media_data["TANK_NO"] == tank) & (media_data["SAMP_NM"] == 4)]["[(NH4)2SO4]"]
        po4 = media_data[(media_data["AUFNR"] == aufnr) & (media_data["TANK_NO"] == tank) & (media_data["SAMP_NM"] == 4)]["KH2PO4"]
        bm = media_data[(media_data["AUFNR"] == aufnr) & (media_data["TANK_NO"] == tank) & (media_data["SAMP_NM"] == 4)]["BM"]
        cm = media_data[(media_data["AUFNR"] == aufnr) & (media_data["TANK_NO"] == tank) & (media_data["SAMP_NM"] == 4)]["CM"]
        sucrose = media_data[(media_data["AUFNR"] == aufnr) & (media_data["TANK_NO"] == tank) & (media_data["SAMP_NM"] == 4)]["Sucrose"]
        betaine = media_data[(media_data["AUFNR"] == aufnr) & (media_data["TANK_NO"] == tank) & (media_data["SAMP_NM"] == 4)]["Betaine"]
        
        df.at[index, 'AGE_1M'] = age
        df.at[index, 'OD_1M'] = od
        df.at[index, 'AN_1M'] = an
        
        df.at[index, '(NH4)2SO4_1M'] = so4
        df.at[index, 'KH2PO4_1M'] = po4
        df.at[index, 'BM_1M'] = bm
        df.at[index, 'CM_1M'] = cm
        df.at[index, 'Sucrose_1M'] = sucrose
        df.at[index, 'Betaine_1M'] = betaine
        
        df.at[index,'ACC_SUGAR_1M'] = float(prev_sug)+float(sug)
              
df[df.isna()['SAMP_NM_1M']]

,AUFNR,TANK_NO,SAMP_NM_START,OD_START,AN_START,JAR_VOL,SAMP_NM_1F,AGE_1F,OD_1F,AN_1F,...,OD_1M,AN_1M,MBO_VOL,(NH4)2SO4_1M,KH2PO4_1M,BM_1M,CM_1M,Sucrose_1M,Betaine_1M,FBO_AGE
4,FFL3J1901005,1,1,16.299999,1.82,3,2,1.333333,22.400000,1.29,...,58.10,1.200,NaN,4.05,0.0,10.470000,0.0,10.710000,0.0,23.62
7,FFL3J1901005,4,1,15.600000,2.02,3,2,1.516667,22.200001,1.10,...,55.98,1.260,NaN,4.05,0.0,10.470000,0.0,10.710000,0.0,26.23
10,FFL3J1902001,3,1,21.230000,1.87,3,2,1.333333,27.299999,0.00,...,71.14,0.602,NaN,4.35,0.0,9.574468,0.0,10.714286,0.0,26.27
93,FLY3J1901001,1,1,17.480000,0.00,3,2,1.416667,22.139999,0.00,...,54.49,2.600,NaN,4.05,0.0,10.470000,0.0,10.710000,0.0,23.85
94,FLY3J1901001,2,1,17.600000,0.00,3,2,1.416667,21.450001,0.00,...,53.15,1.030,NaN,4.05,0.0,10.470000,0.0,10.710000,0.0,25.25
119,FLY5E1810002,1,1,10.990000,4.08,5,2,4.916667,40.110001,1.25,...,62.03,2.140,NaN,16.00,0.2,8.700000,10.3,49.746193,0.0,37.63
120,FLY5E1810002,2,1,11.530000,4.02,5,2,4.966667,40.160000,1.30,...,69.09,0.310,NaN,16.00,0.2,8.700000,10.3,49.746193,0.0,42.83
121,FLY5E1810002,3,1,11.740000,3.95,5,2,5.000000,39.740002,1.15,...,68.14,0.347,NaN,16.00,0.2,8.700000,10.3,49.746193,0.0,34.35
122,FLY5E1810002,4,1,11.400000,4.01,5,2,5.000000,39.970001,1.34,...,69.85,0.250,NaN,16.00,0.2,8.700000,10.3,49.746193,0.0,42.83
123,FLY5E1810002,5,1,11.280000,3.76,5,2,4.600000,37.619999,1.28,...,70.92,0.347,NaN,16.00,0.2,8.700000,10.3,49.746193,0.0,42.83


In [105]:
df['SAMP_NM_1M'].fillna(4,inplace=True)
df[df.isna()['SAMP_NM_1M']]

,AUFNR,TANK_NO,SAMP_NM_START,OD_START,AN_START,JAR_VOL,SAMP_NM_1F,AGE_1F,OD_1F,AN_1F,...,OD_1M,AN_1M,MBO_VOL,(NH4)2SO4_1M,KH2PO4_1M,BM_1M,CM_1M,Sucrose_1M,Betaine_1M,FBO_AGE


In [106]:
df[(df['AUFNR'] == 'FFL3J1901005')][['AUFNR', 'TANK_NO','MBO_VOL']]

,AUFNR,TANK_NO,MBO_VOL
4,FFL3J1901005,1,NaN
5,FFL3J1901005,2,2.6
6,FFL3J1901005,3,2.6
7,FFL3J1901005,4,NaN


MBO_VOL KNN으로 결측치 채우기

In [107]:

x= df
y= df[['MBO_VOL']]

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
X = x.apply(lambda x: encoder.fit_transform(x))
y = y.apply(lambda y: encoder.fit_transform(y))


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)



from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)

print(knn.score(X_test, y_test))

0.8947368421052632


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [108]:

class_cd = pd.Series(encoder.classes_).to_dict()
data_to_be_predicted = df[df.isnull()['MBO_VOL']]
data_to_be_predicted.apply(lambda x: encoder.fit_transform(x))

knn.predict(data_to_be_predicted.apply(lambda x: encoder.fit_transform(x)))
fill_na_values = knn.predict(data_to_be_predicted.apply(lambda x: encoder.fit_transform(x)))
data_to_be_predicted['MBO_VOL'] = fill_na_values
data_to_be_predicted['MBO_VOL']= data_to_be_predicted['MBO_VOL'].replace(class_cd)

df.fillna(data_to_be_predicted, inplace=True)
df.head(10)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,AUFNR,TANK_NO,SAMP_NM_START,OD_START,AN_START,JAR_VOL,SAMP_NM_1F,AGE_1F,OD_1F,AN_1F,...,OD_1M,AN_1M,MBO_VOL,(NH4)2SO4_1M,KH2PO4_1M,BM_1M,CM_1M,Sucrose_1M,Betaine_1M,FBO_AGE
0,FFL3J1901004,1,1,15.940000,1.72,3,2,1.566667,24.580000,1.16,...,57.939999,0.491,3.0,4.05,0.0,10.470000,0.0,10.710000,0.0,22.42
1,FFL3J1901004,2,1,16.110001,1.64,3,2,1.566667,24.780001,1.13,...,59.639999,0.647,2.6,4.05,0.0,10.470000,0.0,10.710000,0.0,21.77
2,FFL3J1901004,3,1,16.209999,1.68,3,2,1.600000,24.410000,1.09,...,59.599998,0.984,2.6,4.05,0.0,10.470000,0.0,10.710000,0.0,32.25
3,FFL3J1901004,4,1,15.820000,1.62,3,2,1.700000,24.629999,1.10,...,62.600000,0.630,2.6,4.05,0.0,10.470000,0.0,10.710000,0.0,26.92
4,FFL3J1901005,1,1,16.299999,1.82,3,2,1.333333,22.400000,1.29,...,58.100000,1.200,3.0,4.05,0.0,10.470000,0.0,10.710000,0.0,23.62
5,FFL3J1901005,2,1,15.900000,1.77,3,2,1.380000,23.400000,1.26,...,58.900002,0.602,2.6,4.05,0.0,10.470000,0.0,10.710000,0.0,22.92
6,FFL3J1901005,3,1,16.299999,1.84,3,2,1.433333,22.900000,1.20,...,58.200001,1.050,2.6,4.05,0.0,10.470000,0.0,10.710000,0.0,21.83
7,FFL3J1901005,4,1,15.600000,2.02,3,2,1.516667,22.200001,1.10,...,55.980000,1.260,3.0,4.05,0.0,10.470000,0.0,10.710000,0.0,26.23
8,FFL3J1902001,1,1,19.820000,1.77,3,2,1.333333,26.400000,0.00,...,63.099998,1.250,2.9,4.35,0.0,9.574468,0.0,10.714286,0.0,21.65
9,FFL3J1902001,2,1,15.970000,0.00,3,2,1.433333,24.200001,0.00,...,63.279999,1.080,2.9,4.35,0.0,9.574468,0.0,10.714286,0.0,23.28


In [109]:
df[(df['AUFNR'] == 'FFL3J1901005')][['AUFNR', 'TANK_NO','MBO_VOL']]

,AUFNR,TANK_NO,MBO_VOL
4,FFL3J1901005,1,3.0
5,FFL3J1901005,2,2.6
6,FFL3J1901005,3,2.6
7,FFL3J1901005,4,3.0


AN_2F Random Forest로 결측치 채우기

In [110]:

x= df
y= df[['AN_2F']]

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
X = x.apply(lambda x: encoder.fit_transform(x))
y = y.apply(lambda y: encoder.fit_transform(y))


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)


from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

forest_reg = RandomForestRegressor()
forest = forest_reg.fit(X_train, y_train)

print(forest.score(X_test, y_test))

0.996683848766032


C:\Users\User\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [111]:

class_cd = pd.Series(encoder.classes_).to_dict()
data_to_be_predicted = df[df.isnull()['AN_2F']]
data_to_be_predicted.apply(lambda x: encoder.fit_transform(x))

forest.predict(data_to_be_predicted.apply(lambda x: encoder.fit_transform(x)))
fill_na_values = knn.predict(data_to_be_predicted.apply(lambda x: encoder.fit_transform(x)))
data_to_be_predicted['AN_2F'] = fill_na_values
data_to_be_predicted['AN_2F']= data_to_be_predicted['AN_2F'].replace(class_cd)

df.fillna(data_to_be_predicted, inplace=True)
df.head()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,AUFNR,TANK_NO,SAMP_NM_START,OD_START,AN_START,JAR_VOL,SAMP_NM_1F,AGE_1F,OD_1F,AN_1F,...,OD_1M,AN_1M,MBO_VOL,(NH4)2SO4_1M,KH2PO4_1M,BM_1M,CM_1M,Sucrose_1M,Betaine_1M,FBO_AGE
0,FFL3J1901004,1,1,15.940000,1.72,3,2,1.566667,24.580000,1.16,...,57.939999,0.491,3.0,4.05,0.0,10.47,0.0,10.71,0.0,22.42
1,FFL3J1901004,2,1,16.110001,1.64,3,2,1.566667,24.780001,1.13,...,59.639999,0.647,2.6,4.05,0.0,10.47,0.0,10.71,0.0,21.77
2,FFL3J1901004,3,1,16.209999,1.68,3,2,1.600000,24.410000,1.09,...,59.599998,0.984,2.6,4.05,0.0,10.47,0.0,10.71,0.0,32.25
3,FFL3J1901004,4,1,15.820000,1.62,3,2,1.700000,24.629999,1.10,...,62.600000,0.630,2.6,4.05,0.0,10.47,0.0,10.71,0.0,26.92
4,FFL3J1901005,1,1,16.299999,1.82,3,2,1.333333,22.400000,1.29,...,58.100000,1.200,3.0,4.05,0.0,10.47,0.0,10.71,0.0,23.62


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 148
Data columns (total 42 columns):
AUFNR            149 non-null object
TANK_NO          149 non-null int64
SAMP_NM_START    149 non-null int64
OD_START         149 non-null float64
AN_START         149 non-null float64
JAR_VOL          149 non-null int64
SAMP_NM_1F       149 non-null int64
AGE_1F           149 non-null float64
OD_1F            149 non-null float64
AN_1F            149 non-null float64
(NH4)2SO4_1F     149 non-null float64
KH2PO4_1F        149 non-null float64
BM_1F            149 non-null float64
CM_1F            149 non-null float64
Sucrose_1F       149 non-null float64
Betaine_1F       149 non-null float64
SAMP_NM_2F       149 non-null int64
ACC_SUGAR_2F     149 non-null float64
AGE_2F           149 non-null float64
OD_2F            149 non-null float64
AN_2F            149 non-null float64
(NH4)2SO4_2F     149 non-null float64
KH2PO4_2F        149 non-null float64
BM_2F            149 non-null fl

In [113]:
df.drop(['JAR_VOL','AUFNR','TANK_NO','SAMP_NM_START','SAMP_NM_1F','SAMP_NM_2F','SAMP_NM_1M'],axis=1, inplace=True)


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 148
Data columns (total 35 columns):
OD_START        149 non-null float64
AN_START        149 non-null float64
AGE_1F          149 non-null float64
OD_1F           149 non-null float64
AN_1F           149 non-null float64
(NH4)2SO4_1F    149 non-null float64
KH2PO4_1F       149 non-null float64
BM_1F           149 non-null float64
CM_1F           149 non-null float64
Sucrose_1F      149 non-null float64
Betaine_1F      149 non-null float64
ACC_SUGAR_2F    149 non-null float64
AGE_2F          149 non-null float64
OD_2F           149 non-null float64
AN_2F           149 non-null float64
(NH4)2SO4_2F    149 non-null float64
KH2PO4_2F       149 non-null float64
BM_2F           149 non-null float64
CM_2F           149 non-null float64
Sucrose_2F      149 non-null float64
Betaine_2F      149 non-null float64
SOD             149 non-null float64
Glucose         149 non-null float64
ACC_SUGAR_1M    149 non-null float64
AGE_1M 

파일로 내보내기

In [115]:
df.to_csv('bio_data_temp.csv',header=True,index= False)

In [116]:
end_time = time.time()
print(start_time)
print(end_time)
print("time :", time.time() - start_time)

1565140479.6094174
1565140487.6718528
time : 8.062435388565063
